# Machine Learning in Business: OilyGiant Mining Location Discovery. 

Project will demonstrate ability to perform machine learning in a business setting.  Data comes from OilyGiant mining company (geo_data_0/1/2.csv which has three feature points). Target task is to find the best place for a new well.

Conditions to Assist in Model. :
Only linear regression is suitable for model training.
A study of 500 points is carried with picking the best 200 points for the profit calculation.
The budget for development of 200 oil wells is 100 USD million.
One barrel of raw materials brings 4.5 USD of revenue The revenue from one unit of product is 4,500 dollars (volume of reserves is in thousand barrels).
After the risk evaluation, keep only the regions with the risk of losses lower than 2.5%. From the ones that fit the criteria, the region with the highest average profit should be selected.

Procedure:
Collect the oil well parameters in the selected region: oil quality and volume of reserves;
Build a model for predicting the volume of reserves in the new wells.  Analyze to pick region with highest profit margin.
Pick the oil wells with the highest estimated values;
Pick the region with the highest total profit for the selected oil wells.
Data is on oil samples from three regions. Parameters of each oil well in the region are already known. Analyze potential profit and risks using the Bootstrapping technique.


In [1]:
#import all needed libraries 
import pandas as pd
import numpy as np

#import display libraries 
import seaborn as sns
import matplotlib.pyplot as plt

#import named regression models 
from sklearn.linear_model import LinearRegression

#import ability to split into training and testing data sets 
from sklearn.model_selection import train_test_split

#import ability to evaluate accuracy of data 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score


from joblib import dump

#needed to compare. 
from sklearn.utils import shuffle
from sklearn.utils import resample
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error



EDA of files

In [2]:
site1=pd.read_csv('/datasets/geo_data_0.csv')
site1.head(3)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647


In [3]:
site1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
#check for empty column values
site1.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [5]:
#check for duplicates 
print(site1['id'].duplicated().sum())

10


In [6]:
#confirm correct column names 
site1.columns

Index(['id', 'f0', 'f1', 'f2', 'product'], dtype='object')

In [7]:
# Drop duplicates in the DataFrame itself
site1 = site1.drop_duplicates(subset=['id'])

# Confirm duplicates were dropped
print(site1['id'].duplicated().sum())  

0


In [8]:
print(site1.shape)
print(site1.columns)

(99990, 5)
Index(['id', 'f0', 'f1', 'f2', 'product'], dtype='object')


In [9]:
site2=pd.read_csv('/datasets/geo_data_1.csv')
site2.head(3)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305


In [10]:
site2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [11]:
print(site2.head(3))

      id         f0        f1        f2     product
0  kBEdx -15.001348 -8.276000 -0.005876    3.179103
1  62mP7  14.272088 -3.475083  0.999183   26.953261
2  vyE1P   6.263187 -5.948386  5.001160  134.766305


In [12]:
site2.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [13]:
site2.duplicated().sum()

0

In [14]:
site2.shape

(100000, 5)

In [15]:
site3=pd.read_csv('/datasets/geo_data_2.csv')
site3.head(3)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910


In [16]:
site3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [17]:
site3.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [18]:
site3.duplicated().sum()

0

In [19]:
site3.shape

(100000, 5)

3. Profit calculation prep

In [20]:
sites=[site1,site2,site3]


for i in range(3):
    sites[i] = sites[i].drop(['id'],axis=1)
    display(sites[i])


Confirmed that cleared each dataframe of "id" column. 

In [21]:
#define variables from source data 
revenue_per_thousand_barrels = 100000000 
revenue_per_well = 4500 
wells=200

# Calculate break-even volume
break_even_volume = revenue_per_thousand_barrels/revenue_per_well

#calculate average break-even volume for each well
average_vol_each_well= break_even_volume/wells

print(f'Break-even volume needed: {break_even_volume}')
print(f'Average volume required per well to break-even: {average_vol_each_well}')



Break-even volume needed: 22222.222222222223
Average volume required per well to break-even: 111.11111111111111


4. Write functions to calculate profit, bootstrap, and rmse from a set of selected oil wells and model predictions:

<div class="alert alert-block alert-success">
<b>Reviewer's comment</b> <a class="tocSkip"></a>
Success. This snippet accurately calculates the break-even volume and the average volume required per well to break even. The use of constants like revenue_per_thousand_barrels and revenue_per_well ensures clarity and maintains scalability for future changes.
</div>


In [22]:
def profit_calculation(target, predictions, count):
    # Sort predictions in descending order
    probs_sorted = predictions.sort_values(ascending=False)    
    # Select top 'count' values using valid indices
    selected = target[probs_sorted.index][:count]
    return revenue_per_well * selected.sum() - revenue_per_thousand_barrels


In [23]:
#rmse calculation 
def calculate_rmse(true_values, predicted_values):
    return np.sqrt(mean_squared_error(true_values, predicted_values))

In [24]:
#from 
def bootstrap_profit(valid, predictions, wells, profit_calculation, n_samples=1000, random_state=None):
    values = []
    assert (valid.index == predictions.index).all()
    for _ in range(n_samples):
        target_ranked = valid['product'].sample(n=500, replace=True, random_state=12345)
        probability_ranked = predictions.iloc[target_ranked.index]      
        values.append(profit_calculation(target_ranked, probability_ranked, wells))

    values = pd.Series(values)
    return {
        'mean': values.mean(),
        'low_quantile': values.quantile(0.025),
        'high_quantile': values.quantile(0.975),
        'risk_of_loss': (values < 0).mean() * 100
    }

In [25]:
#profit computation 
def calculate_profit(valid, predictions, wells, profit_calculation):
    return profit_calculation(valid['product'], predictions, wells)

5. Calculate risk and profit for each region 

In [26]:
for i, site in enumerate(sites):
    #drop unnecessary column 'id'
    site = site.drop(columns=['id'])
    display(site)
    
    train, valid = train_test_split(site, test_size=0.25, random_state=12345)
    train = train.reset_index()
    valid = valid.reset_index()

    # Train model
    model = LinearRegression()
    model.fit(train.drop(['product'], axis=1), train['product'])
    predictions = pd.Series(model.predict(valid.drop(['product'], axis=1)), index=valid.index)
    
    # assert valid.index == predictions.index

    print(f'Information for Site {i}')

    # Calculate and display RMSE
    rmse = calculate_rmse(valid['product'], predictions)
    print(f'RMSE:', rmse)

    # Average predicted reserves
    average = predictions.mean()
    print(f'Predicted Reserves Average Volume:', average)

    # Calculate and display profit
    profit = calculate_profit(valid, predictions, wells, profit_calculation)
    print(f'Profit: {profit}')

    # Bootstrap analysis
    stats = bootstrap_profit(valid, predictions, wells, profit_calculation, n_samples=1000, random_state=12345)
    print(f'Average profit:', stats['mean'])
    print(f'2.5% quantile:', stats['low_quantile'])
    print(f'97.5% quantile:', stats['high_quantile'])
    print(f'Risk of loss:', stats['risk_of_loss'], '%')
    print('******************************************************************')
    print()


,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...
99995,0.971957,0.370953,6.075346,110.744026
99996,1.392429,-0.382606,1.273912,122.346843
99997,1.029585,0.018787,-1.348308,64.375443
99998,0.998163,-0.528582,1.583869,74.040764


Information for Site 0
RMSE: 37.853643189399115
Predicted Reserves Average Volume: 92.7894653773364
Profit: 33651872.37700285
Average profit: 7513523.9304731935
2.5% quantile: 7513523.9304731935
97.5% quantile: 7513523.9304731935
Risk of loss: 0.0 %
******************************************************************



,f0,f1,f2,product
0,-15.001348,-8.276000,-0.005876,3.179103
1,14.272088,-3.475083,0.999183,26.953261
2,6.263187,-5.948386,5.001160,134.766305
3,-13.081196,-11.506057,4.999415,137.945408
4,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...
99995,9.535637,-6.878139,1.998296,53.906522
99996,-10.160631,-12.558096,5.005581,137.945408
99997,-7.378891,-3.084104,4.998651,137.945408
99998,0.665714,-6.152593,1.000146,30.132364


Information for Site 1
RMSE: 0.8930659052625232
Predicted Reserves Average Volume: 68.72849982676388
Profit: 24150866.966815114
Average profit: 2993593.720942274
2.5% quantile: 2993593.7209422737
97.5% quantile: 2993593.7209422737
Risk of loss: 0.0 %
******************************************************************



,f0,f1,f2,product
0,-1.146987,0.963328,-0.828965,27.758673
1,0.262778,0.269839,-2.530187,56.069697
2,0.194587,0.289035,-5.586433,62.871910
3,2.236060,-0.553760,0.930038,114.572842
4,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...
99995,-1.777037,1.125220,6.263374,172.327046
99996,-1.261523,-0.894828,2.524545,138.748846
99997,-1.199934,-2.957637,5.219411,157.080080
99998,-2.419896,2.417221,-5.548444,51.795253


Information for Site 2
RMSE: 40.03003812274246
Predicted Reserves Average Volume: 94.96522806982759
Profit: 26887004.60222158
Average profit: -950441.7713700086
2.5% quantile: -950441.7713700086
97.5% quantile: -950441.7713700086
Risk of loss: 100.0 %
******************************************************************



Project Summary 

Cleaning of each dataframe had to be performed independently.  Once completed tasks could be grouped together.  Each dataframe needed the "id" column dropped before running through the model.  All other tasks were able to be placed in a loop where each dataframe was looped through. 

In order to break-even 22,222.22 barrels of oil need to be generated at a site.  Average volume required per well to break-even is 111.11 barrells. 

Site 1 has the lowest risk of loss among all the evaluated sites.  Site 1 has the largest average profit at 5 million dollars.  Site 1 is also the only site where the lower quantile is a positive number.  It is notebwrothy that Site 1 has the lowest predicted average volume of reserves by almost 30 barrels.  However Site 1 remains the best business decision because its extreme low risk of failure.  The expected 24 million dollars generation from this site could fund exploration of sites besides site 0 and site 2 for Oily Giant.   